# Методы предобучения без учителя, ФКН ВШЭ

## Домашнее задание 3: предобучение языковых моделей

### Оценивание и штрафы

Максимально допустимая оценка за работу — **13** баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании мы реализуем три (четыре) метода предобучения языковых моделей и сравним их качество на трех downstream задачах. Мы будем работать с моделью T5-small, имеющую архитектуру энкодер-декодер. Для предобучения будем использовать датасет [BookCorpus](https://huggingface.co/datasets/bookcorpus) (точнее его десятую часть, чтобы было реально обучиться за ограниченное время). Этот датасет содержит 74 миллиона предложений из книг и использовался для предобучения практически всех моделей, так что нам он идеально подойдет.  

Как и в прошлой домашке, для сдачи задания необходимо провести эксперименты, описанные в ноутбуке, и написать о них отчет в формате PDF. Вместе с отчетом сдается код, позволяющий запустить эксперименты. Прежде чем что-то имплементировать, прочитайте все постановки экспериментов и подумайте, как лучше организовать код, не забудьте также зачекпойнтить необходимые обученные модели. Мы оставляем за собой право снижать оценку за плохо структурированный код. **Обязательно** используйте оптимизации для обучения из [списка](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html) в своих пайплайнах, чтобы ускорить запуски. Особенно Automatic Mixed Precision. Также **обратите внимание**, что отчет &mdash; это обязательная составляющая оценки, без него мы не будем проверять ваше задание. Отчет обязательно должен включать кривые обучения для всех моделей, которые вы запускаете. Кроме того, следите за тем, чтобы ваши рисунки и графики были читаемыми, и по возможности красивыми :)

При тестировании наших методов предобучения мы будем проводить замеры, дообучая модель на трех задачах по отдельности:

* Ответы на вопросы: датасет [SQuAD](https://huggingface.co/datasets/squad), метрики **ExactMatch (EM)** и **F1**.
* Суммаризация: датасет [XSum](https://huggingface.co/datasets/xsum), метрика **ROUGE-1**.
* Машинный перевод: **сотая часть** датасета [WMT-16 [de-en]](https://huggingface.co/datasets/wmt16), метрика **BLEU**.

Код для дообучения вы можете написать сами, а можете ориентироваться на ноутбуки, прикрепленные к заданию. Так же можно использовать официальные туториалы от huggingface ([ответы на вопросы](https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb), [суммаризация](https://github.com/huggingface/notebooks/blob/main/examples/summarization.ipynb), [перевод](https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb)). Туториалы реализованы через специализированные классы, что удобно, однако работают они существенно медленнее.


## Задание 0. Supervised baseline

**0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов**

Как обычно, перед исследованием задач предобучения построим бейзлайн с помощью модели, обученной supervised из случайного начального приближения. Для этого задания и всех остальных далее зафиксируем архитектуру T5-small. Гиперпараметры для обучения разрешается выбрать любыми разумными, главное --сохраняйте их неизменными для всех заданий, чтобы сравнение было валидным. **Обратите внимание**, что в задаче ответов на вопросы от модели требуется только энкодер.

In [ ]:
from transformers import T5ForConditionalGeneration, T5Config
model = T5ForConditionalGeneration(T5Config())

## Задание 1. MLM

**3 балла**

Реализуйте и обучите метод Masked Language Model (MLM) из статьи [Devlin et al, 2018](https://arxiv.org/pdf/1810.04805.pdf). В этом задании запрещается заимствовать код из открытых источников. Параметры для маскирования следует взять такими же, как в статье. Для обучения используйте **десятую часть** датасета BookCorpus. Длину всех текстов ограничим значением 256. Обучение скорее всего будет занимать больше часа на одну эпоху, поэтому для экспериментов советуем взять небольшую подвыборку. При обучении на всей выборке 2-3 эпох должно быть достаточно, чтобы получить улучшение над supervised моделью.

При дообучении на seq2seq задачи декодер придется инициализировать случайно.



In [ ]:
from datasets import load_dataset

dataset = load_dataset("bookcorpus", split="train", num_proc=8)
dataset = dataset.select(range(int(len(dataset) * 0.1)))

In [ ]:
from transformers import T5TokenizerFast

tokenizer = T5TokenizerFast.from_pretrained("t5-small")

In [ ]:
encoded_dataset = dataset.map(lambda examples: tokenizer(examples['text'], max_length=256, batched=True))

In [ ]:
encoded_dataset.save_to_disk("encoded_bookcorpus")

При создании батчей очень важно тратить как можно меньше места на паддинги, при этом полностью заполняя память. Для этого размер батча удобнее измерять числом токенов, а не количеством текстов. Тогда в каждом батче будут лежать тексты примерно одной длины так, чтобы число токенов, включая паддинги, было примерно фиксированным. К сожалению, хорошей реализации такого семплирования я никогда не видел, поэтому ниже мой вариант.

In [ ]:
import random
from torch.utils.data import Sampler
from copy import copy


class FixedTokensSampler(Sampler):
    def __init__(
        self,
        dataset: datasets.arrow_dataset.Dataset,
        n_tokens: int,
        lengths: np.array = None,
        max_len: int = 256,
        shuffle: bool = True
    ):
        self.n_tokens = n_tokens
        self.shuffle = shuffle
        
        if lengths is not None:
            self.lengths = lengths
        else:
            # default iteration over a dataset is too slow
            self.lengths = np.array([(i, len(sample)) for i, sample in enumerate(tqdm(dataset._data['input_ids']))])

    def __iter__(self):
        if self.shuffle:
            # fast shuffle
            self.lengths = self.lengths[np.random.permutation(len(self.lengths))]

        mean_length = np.mean(self.lengths, axis=0)[1]
        
        # approximatelly 100 batches of with mean-sized samples
        step = int(self.n_tokens / mean_length * 100)
        for i in range(0, len(self.lengths), step):
            pooled = sorted(self.lengths[i:i + step], key=lambda x: x[1])

            batches = []
            batch = []
            cur_n_tokens = 0
            for idx, length in pooled:
                # lengths are sorted in ascending order
                if (len(batch) + 1) * length > self.n_tokens:
                    if len(batch) == 0:
                        print(f'Maximum number of tokens {self.n_tokens} is lower, than size of a sample {length}')
                        break
                    else:
                        batches.append(copy(batch))
                        batch = []
                        cur_n_tokens = 0
                else:
                    batch.append(idx)
                    cur_n_tokens += length
            
            if self.shuffle:
                random.shuffle(batches)

            for batch in batches:
                yield batch

    def __len__(self):
        # approximate size (lower bound)
        return np.sum(self.lengths, axis=0)[1] // self.n_tokens


def collate_batch(batch, pad_id=0):
    """
    this function recieves batch and returns its modified version
    """
    input_ids = []
    for sample in batch:
        input_ids.append(torch.tensor(sample['input_ids'])) 

    return pad_sequence(input_ids, padding_value=pad_id, batch_first=True)

In [ ]:
sampler = FixedTokensSampler(encoded_dataset, n_tokens=..., shuffle=True)

dataloader = torch.utils.data.DataLoader(
    encoded_dataset,
    batch_sampler=sampler,
    collate_fn=collate_batch,
    num_workers=16
)

## Задание 2. GPT

**3 балла**

Аналогично предыдущему заданию, реализуйте language modeling задачу предобучения для декодера как это было сделано в статье про GPT ([Radford et al, 2018](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)). Как изменилось качество на downstream задачах относительно MLM? Почему так?

## Задание 3. MASS

**4 балла**

Реализуйте метод MASS ([Song et al, 2019](https://arxiv.org/pdf/1905.02450.pdf)) для предобучение энкодера и декодера с помощью демаскирования последовательностей токенов. В оригинальной статье предлагается маскировать только одну последовательность, однако в подходе T5 ([Raffel et al, 2020](https://arxiv.org/pdf/1910.10683.pdf)) предлагается маскировать несколько последовательностей небольшой длины. Оба способа должны показать прирост в метриках при дообучении на seq2seq задачах. Проверьте, какой способ лучше и прокомментируйте результаты.

## Бонус 1. BART

**3 балла**

Аналогично предыдущим заданиям реализуйте метод BART ([Lewis et al, 2019](https://arxiv.org/pdf/1910.13461.pdf)) и замерьте качество. Не обязательно брать все 5 функциналов для обучения, трех на ваш выбор будет достаточно.

В статье предлагается использовать случайно инициализированный энкодер для задачи машинного перевода, чтобы работать с языками, отличными от предобучающих. Проверьте на нашем датасете для перевода, есть ли в этом смысл, обучив модель переводить с немецкого на английский.